This notebook uses auto-regression techniques to study the behavior in the calibration value.

In [1]:
#import the stuff
import pandas as pd #dataframes etc
import matplotlib.pyplot as plt #plotting
import numpy as np
from sklearn import preprocessing
from common.utils import TimeSeriesTensor, create_evaluation_df, mape, scale_shrinker
#now lets try some autoregression
import seaborn as sns
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics

In [4]:
xtal_list = [54003,54004,54005,54006,54007,54008,54009]
#preload test data
test_xtals = {}
results = {}
for xtal in xtal_list:
    suffix = '_{}.csv'.format(xtal)
    test = pd.read_csv('../data/test_diff'+suffix, index_col=0)
    
    test.index = pd.to_datetime(test.index)
    test = test[['p2']]
    
    test_xtals[xtal] = test
for xtal in xtal_list:
    #load the data###################################
    #the other xtals
    sub_list = xtal_list.copy()
    sub_list.remove(xtal)
    #
    print('Loading xtal: {}'.format(xtal))
    valid = pd.read_csv('../data/valid_diff'+suffix, index_col=0)
    train = pd.read_csv('../data/train_diff'+suffix, index_col=0)
    #set index to datetime
    
    valid.index = pd.to_datetime(valid.index)
    train.index = pd.to_datetime(train.index)
    #only take calibration values
    
    valid = valid[['p2']]
    train = train[['p2']]
    #full data
    full = train.append([valid,test])
    full.head()
    #lets pool the valid and train data together
    train = train.append(valid)

    #shape input data#################################
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    train['p2'] = scaler.fit_transform(train)[:,0]
    train.head()

    test = test_xtals[xtal]
    test['p2'] = scaler.transform(test)

    #Number of steps to forecast ahead
    test_shifted = test.copy()
    HORIZON = 3
    #create test data points for each horizon
    for t in range(1, HORIZON):
        test_shifted['p2+'+str(t)] = test_shifted['p2'].shift(-t, freq='H')

    test_shifted = test_shifted.dropna(how='any')
    test_shifted.head(5)

    #make predictions on the test data#################
    training_window = 500
    train_series = train['p2']
    test_series = test_shifted
    lag = 10

    history = [x for x in train_series]
    history = history[(-training_window):]
    predictions = list()
    for t in range(test_series.shape[0]):
        model = AutoReg(history, lag, old_names=False)
        model_fit = model.fit()
        yhat = model_fit.forecast(steps=HORIZON)
        predictions.append(yhat)
        obs = list(test_series.iloc[t])

        # move the training window
        history.append(obs[0])
        history.pop(0)
        #print(test_series.index[t])
        #print(t+1, ': predicted =', yhat, 'expected =', obs)
    pred_np = np.array(predictions)
    #calculate MAPE
    mape1 = mape(pred_np[:,0], test_series['p2'])
    mape2 = mape(pred_np[:,1], test_series['p2+1'])
    mape3 = mape(pred_np[:,2], test_series['p2+2'])
    print('Evaluating AR({}) on same xtal:'.format(lag))
    print("\tMAPE prediction 1 hour  ahead: {:.1%}".format(mape1),
              "\n\tMAPE prediction 2 hours ahead: {:.1%}".format(mape2),
              "\n\tMAPE prediction 3 hours ahead: {:.1%}".format(mape3))
    my_mapes = [mape1, mape2, mape3]
    ####LOOP OVER THE OTHER CRYSTAL DATA AND TEST AGAIN
    other_mape1 = []
    other_mape2 = []
    other_mape3 = []
    for o_xtal in sub_list:
        print('\tPredicting on: {}'.format(o_xtal))
        test = test_xtals[o_xtal]
        test['p2'] = scaler.transform(test)

        #Number of steps to forecast ahead
        test_shifted = test.copy()
        HORIZON = 3
        #create test data points for each horizon
        for t in range(1, HORIZON):
            test_shifted['p2+'+str(t)] = test_shifted['p2'].shift(-t, freq='H')

        test_shifted = test_shifted.dropna(how='any')
        test_shifted.head(5)

        #make predictions on the test data#################
        training_window = 500
        train_series = train['p2']
        test_series = test_shifted
        lag = 10

        history = [x for x in train_series]
        history = history[(-training_window):]
        predictions = list()
        for t in range(test_series.shape[0]):
            model = AutoReg(history, lag, old_names=False)
            model_fit = model.fit()
            yhat = model_fit.forecast(steps=HORIZON)
            predictions.append(yhat)
            obs = list(test_series.iloc[t])

            # move the training window
            history.append(obs[0])
            history.pop(0)
            #print(test_series.index[t])
            #print(t+1, ': predicted =', yhat, 'expected =', obs)
        pred_np = np.array(predictions)
        #calculate MAPE
        mape1 = mape(pred_np[:,0], test_series['p2'])
        mape2 = mape(pred_np[:,1], test_series['p2+1'])
        mape3 = mape(pred_np[:,2], test_series['p2+2'])
        print('\t\tEvaluating AR({}) on {}:'.format(lag,o_xtal))
        print("\t\tMAPE prediction 1 hour  ahead: {:.1%}".format(mape1),
                  "\n\t\tMAPE prediction 2 hours ahead: {:.1%}".format(mape2),
                  "\n\t\tMAPE prediction 3 hours ahead: {:.1%}".format(mape3))
        other_mape1.append(mape1)
        other_mape2.append(mape2)
        other_mape3.append(mape3)
    print(len(other_mape1))
    print(len(other_mape2))
    print(len(other_mape3))
    #now let's average the other mapes
    other_mape1_avg = sum(other_mape1)/len(other_mape1)
    other_mape2_avg = sum(other_mape2)/len(other_mape2)
    other_mape3_avg = sum(other_mape3)/len(other_mape3)
    print("\n\tAVG other MAPE prediction 1 hour  ahead: {:.1%}".format(other_mape1_avg),
                  "\n\tMAPE prediction 2 hours ahead: {:.1%}".format(other_mape2_avg),
                  "\n\tMAPE prediction 3 hours ahead: {:.1%}".format(other_mape3_avg))
    my_mapes.append([other_mape1_avg,other_mape2_avg,other_mape3_avg])
    results[xtal] = tuple(my_mapes)
    
    
#now let's summarize everything
results_df = pd.DataFrame.from_dict(results, orient='index',columns=['MAPE1', 'MAPE2', 'MAPE3', 'other_MAPE1', 'other_MAPE2', 'other_MAPE3'])
results_df.head(10)

Loading xtal: 54003
Evaluating AR(10) on same xtal:
	MAPE prediction 1 hour  ahead: 2.6% 
	MAPE prediction 2 hours ahead: 3.9% 
	MAPE prediction 3 hours ahead: 4.8%
	Predicting on: 54004
		Evaluating AR(10) on 54004:
		MAPE prediction 1 hour  ahead: 2.5% 
		MAPE prediction 2 hours ahead: 3.6% 
		MAPE prediction 3 hours ahead: 4.3%
	Predicting on: 54005
		Evaluating AR(10) on 54005:
		MAPE prediction 1 hour  ahead: 2.5% 
		MAPE prediction 2 hours ahead: 3.2% 
		MAPE prediction 3 hours ahead: 3.7%
	Predicting on: 54006
		Evaluating AR(10) on 54006:
		MAPE prediction 1 hour  ahead: 2.8% 
		MAPE prediction 2 hours ahead: 4.0% 
		MAPE prediction 3 hours ahead: 4.8%
	Predicting on: 54007
		Evaluating AR(10) on 54007:
		MAPE prediction 1 hour  ahead: 2.7% 
		MAPE prediction 2 hours ahead: 3.7% 
		MAPE prediction 3 hours ahead: 4.4%
	Predicting on: 54008
		Evaluating AR(10) on 54008:
		MAPE prediction 1 hour  ahead: 2.7% 
		MAPE prediction 2 hours ahead: 3.7% 
		MAPE prediction 3 hours ahead: 

KeyboardInterrupt: 